In [1]:
import pandas as pd
import re

In [2]:
year = '2019'

In [3]:
ccd_names = {'Atlético Ottawa' : 'Atlético Ottawa',
            'Cavalry' : 'Cavalry FC',
            'Edmonton' : 'FC Edmonton',
            'Forge' : 'Forge FC',
            'HFX Wanderers' : 'HFX Wanderers FC',
            'Pacific' : 'Pacific FC',
            'Valour' : 'Valour FC',
            'York United' : 'York United FC',
            'York United' : 'York9 FC','York9' : 'York9 FC'}

In [4]:
team_names = {'Atlético Ottawa' : 'ato',
              'Cavalry FC' : 'cav','Cavalry' : 'cav',
              'FC Edmonton' : 'fce','Edmonton' : 'fce',
              'Forge FC' : 'for','Forge' : 'for',
              'HFX Wanderers FC' : 'hfx','HFX Wanderers' : 'hfx',
              'Pacific FC' : 'pac','Pacific' : 'pac',
              'Valour FC' : 'val','Valour' : 'val',
              'York United FC' : 'yor','York United' : 'yor',
              'York9 FC' : 'y9','York9' : 'y9'}

In [5]:
item = 'y9'
for key, value in team_names.items():
    if item in value:
        print(key)
        break

York9 FC


In [6]:
results = pd.read_csv(f'datasets/{year}/league/{year}-results.csv')
try:
    week = results[results['hr'] == 'E']['w'].head(1).values[0]
    schedule = results[results['w'] == week].copy()
    schedule = schedule.fillna(0)
    x = 'This Weeks'
except:
    week = results['w'].tail(1).values[0]
    schedule = results[results['w'] == week].copy()
    x = 'Final Week'

In [7]:
print(f'{x} results:')

Final Week results:


In [8]:
schedule

,game,w,s,d,m,hs,as,home,hr,w.1,...,away,ar,w.2,d.2,l.1,csh,csa,combined,venue,links
99,I100,26,3,2,11,0,1,Cavalry FC,L,0,...,Forge FC,W,1,0,0,0,1,11-2-2019 Cavalry FC L 0-1 W Forge FC,ATCO Field,https://canpl.ca/matchcentre/7f2xl06opfb8koy1j...


In [9]:
import cpl_main as cpl

In [10]:
results_brief = cpl.get_results_brief(results)

In [11]:
results_brief.head()

,d,m,hs,as,home,hr,away,ar,summary,team
0,4,5,2,1,Cavalry FC,W,York9 FC,L,W h 2 1 y9,Cavalry FC
1,8,5,1,0,Cavalry FC,W,Valour FC,L,W h 1 0 val,Cavalry FC
2,18,5,1,0,Cavalry FC,W,FC Edmonton,L,W h 1 0 fce,Cavalry FC
3,25,5,2,0,Cavalry FC,W,HFX Wanderers FC,L,W h 2 0 hfx,Cavalry FC
4,22,6,0,1,Cavalry FC,L,Forge FC,W,L h 0 1 for,Cavalry FC


In [12]:
player_totals = pd.read_csv(f'datasets/{year}/ccd/CPL Player Totals {year}.csv')
team_totals = pd.read_csv(f'datasets/{year}/ccd/CPL Team Totals {year}.csv')
player_game = pd.read_csv(f'datasets/{year}/ccd/CPL Player By Game {year}.csv')
team_game = pd.read_csv(f'datasets/{year}/ccd/CPL Team By Game {year}.csv')

In [13]:
team_ref_all = pd.read_csv('datasets/teams.csv')
team_ref = team_ref_all[team_ref_all['year'] == int(year)]
player_info = pd.read_csv(f'datasets/{year}/player-{year}-info.csv')
season = pd.read_csv(f'datasets/{year}/season.csv')

In [14]:
player_game['name'] = player_game['firstName']+' '+player_game['lastName']

In [22]:
'Win' in team_totals.columns

False

In [21]:
team_totals.groupby('team').sum()

,Rank,GM,optaTeamId,SubOff,SubOn,GWG,SOG,SOTInBox,SOTOBox,BlkdInBox,...,DefTouchD3,DefTouchM3,DefTouchA3,Open Play xG,Set Piece XG,NonPenxG,Key Passes,Defensive Blocks,xG Per Shot,Shots Total
team,,,,,,,,,,,,,,,,,,,,,
Cavalry,3,30,15381,84,84,19,147,109,38,54,...,855,367,107,36.80,23.74,55.81,228,67,0.16,390
Edmonton,5,28,6925,81,81,8,104,69,35,35,...,885,317,62,17.16,15.54,31.91,189,77,0.11,291
Forge,1,30,15377,88,88,19,144,105,39,42,...,970,409,113,31.37,16.40,42.25,225,72,0.13,355
HFX Wanderers,7,28,15376,69,69,6,107,73,34,38,...,1079,344,63,17.10,13.99,28.72,205,80,0.10,318
Pacific,4,28,15380,81,81,8,111,67,44,30,...,926,382,92,23.69,12.22,32.76,204,85,0.12,312
Valour,6,28,15378,82,82,8,116,70,46,48,...,892,311,60,23.18,13.00,31.45,225,86,0.10,358
York United,2,28,15382,78,78,9,152,102,50,61,...,829,369,75,27.02,16.38,37.09,282,93,0.11,412


### Match similar names and add to player_game

In [15]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

add_names = []
for x in player_game['name'].unique():
    lst, names = [], []
    for y in player_info['name'].unique():
        sim = similar(x,y)
        lst.append(sim)
        names.append(y)
    max_value = max(lst)
    max_index = lst.index(max_value)
    #print(x,' :',names[max_index])
    add_names.append(names[max_index])

#### Need to match players BETWEEN player_totals and player_game

In [16]:
player_game['display'] = 'a'
for i in range(player_game.shape[0]):
    x = player_game.at[i,'name']
    lst, names = [], []
    for y in player_info['display'].unique():
        sim = similar(x,y)
        lst.append(sim)
        names.append(y)
    max_value = max(lst)
    max_index = lst.index(max_value)
    #print(x,'\n',names[max_index])
    player_game.at[i,'display'] = names[max_index]

In [17]:
player_game[['name','display']].head()

,name,display
0,Daniel Krutzen,Daniel Krutzen
1,Alexander Gonzalez Moreno,Alexander González
2,Bertrand Owundi,Bertrand Owundi
3,Diyaeddine Abzi,Diyaeddine Abzi
4,Kwame Awuah,Kwame Awuah


### GET PLAYER STATS

### get player line chart data

position_dict = {'dc':['team','name','Min','Goal','GoalDirFK','GoalHead','Clrnce','Int','SucflDuels','DuelLs','TcklMade%'],
                 'fc':['team','name','Min','ExpG','ExpA','Goal','PenGoal','GoalDirFK','GoalHead','Ast','PsAtt','PsOnHfScs','PsOpHfScs','BgChnc','BgChncCrtd','ChncOpnPl','Chance','SOG','SOTInBox','SOTOBox','ShotDirFK','SOTHead'],
                 'mc':['team','name','Min','ExpG','ExpA','Goal','PenGoal','GoalDirFK','GoalHead','Ast','PsAtt','PsOnHfScs','PsOpHfScs','Touches','DuelLs','SucflDuels','TcklMade%'],
                 'gc':['team','name','Min','CleanSheet','CrossPunch','Saves','SvInBox','SvOutBox','SvPrdDng','SvPrdSaf','SvReach','SvStoop','KeeperCtch','GoalCncd']}
                 
 ### NEWER
 
 position_dict = {'dc':['team','name','Min','Goal','PsOpHfFl','PsOnHfFl','PsAtt','PsOnHfScs','SucflDuels','Touches','DuelLs','SucflTkls','Int','DefTouch','Clrnce','Recovery','TchsD3','TchsM3','PsAttD3','PsAttM3','PsDist','Aerials'],
                 'fc':['team','name','Min','ExpG','ExpA','Goal','SubOn','PsOpHfFl','PsAtt','PsOpHfScs','ChncOpnPl','SucflDuels','Touches','DuelLs','Recovery','FlComA3','FlComM3','FlSufM3','BgChncFace','Disposs','TchsA3','TchsM3','TouchOpBox','PsCmpA3','PsDist'],
                 'mc':['team','name','Min','ExpG','ExpA','Goal','Off','ShotRF','PsOpHfFl','PsOnHfFl','PsAtt','PsOnHfScs','PsOpHfScs','SucflDuels','Touches','DuelLs','DefTouch','Recovery','BgChncFace','TchsA3','TchsD3','TchsM3','PsCmpM3','PsCmpA3','PsAttD3','PsAttM3','PsDist'],
                 'gc':['team','name','Min','CleanSheet','PsOpHfFl','PsOnHfFl','PsAtt','PsOnHfScs','PsOpHfScs','Touches','Recovery','GoalCncd','GoalCnIBx','Saves','SvClct','SvDive','SvHands','SvInBox','SvOutBox','SvPrdSaf','SvStand','BgChncFace','TchsD3','PsCmpD3','PsAttD3','PsDist','ExpGAg']}

In [18]:
position_dict = {'dc':['team','name','Min','ExpG','ExpA','Goal','PsOnHfFl','SucflDuels','Touches','DuelLs','SucflTkls','Int','DefTouch','Clrnce','Recovery','TchsM3','Aerials'],
                 'fc':['team','name','Min','ExpG','ExpA','Goal','Off','PsOpHfFl','PsAtt','PsOpHfScs','ChncOpnPl','SucflDuels','Touches','DuelLs','Recovery','FlComA3','FlSufM3','Disposs','TchsA3','TouchOpBox','PsCmpA3'],
                 'mc':['team','name','Min','ExpG','ExpA','Goal','Off','PsOpHfFl','PsAtt','PsOnHfScs','PsOpHfScs','SucflDuels','Touches','DuelLs','DefTouch','Recovery','TchsA3','TchsM3','PsCmpM3','PsCmpA3'],
                 'gc':['team','name','Min','CleanSheet','PsOnHfFl','Recovery','GoalCncd','GoalCnIBx','Saves','SvClct','SvDive','SvHands','SvInBox','SvOutBox','SvPrdSaf','SvStand','BgChncFace','ExpGAg']}

In [19]:
#len(position_dict['fc']),position_dict['fc']

In [20]:
line_chart_dict = {'dc' : [], 
                   'fc' : [], 
                   'mc' : [], 
                   'gc' : []}
for name in player_game['display'].unique():
    try:
        position = player_info[player_info['display'] == name]['position'].values[0]+'c'
        line_chart_dict[position].append(player_game[player_game['display'] == name][position_dict[position]])
    except:
        try:
            position = player_info[player_info['name'] == name]['position'].values[0]+'c'
            line_chart_dict[position].append(player_game[player_game['display'] == name][position_dict[position]])
        except:
            print('MISSING: ',name)

In [21]:
def add_display_name(data):
    data = cpl.index_reset(data)
    data['display'] = 'a'
    for i in range(data.shape[0]):
        x = data.at[i,'name']
        lst, names = [], []
        for y in player_info['display'].unique():
            sim = similar(x,y)
            lst.append(sim)
            names.append(y)
        max_value = max(lst)
        max_index = lst.index(max_value)
        #print(x,'\n',names[max_index])
        data.at[i,'display'] = names[max_index]
    return data

In [22]:
def add_position_number_team(data):
    
    data = cpl.index_reset(data)
    data['position'] = 'n'
    data['number'] = '0'
    data['team'] = '-'
    for i in range(data.shape[0]):
        try:
            data.at[i,'position'] = player_info[player_info['name'] == data.at[i,'name']]['position'].values[0]
        except:
            try:
                data.at[i,'position'] = player_info[player_info['display'] == data.at[i,'name']]['position'].values[0]
            except:
                data.at[i,'position'] = '-'
        try:
            data.at[i,'number'] = player_info[player_info['name'] == data.at[i,'name']]['number'].values[0]
        except:
            try:
                data.at[i,'number'] = player_info[player_info['display'] == data.at[i,'name']]['number'].values[0]
            except:
                data.at[i,'number'] = '999'
        try:
            data.at[i,'team'] = player_info[player_info['name'] == data.at[i,'name']]['team'].values[0]
        except:
            try:
                data.at[i,'team'] = player_info[player_info['display'] == data.at[i,'name']]['team'].values[0]
            except:
                data.at[i,'team'] = 'No Club FC'
            
    return data

In [23]:
defenders_line = pd.DataFrame()
for i in line_chart_dict['dc']:
    defenders_line = pd.concat([defenders_line,i])
defenders_line = add_display_name(defenders_line)
defenders_line.head()

,team,name,Min,ExpG,ExpA,Goal,PsOnHfFl,SucflDuels,Touches,DuelLs,SucflTkls,Int,DefTouch,Clrnce,Recovery,TchsM3,Aerials,display
0,Forge,Daniel Krutzen,90,0.05,0.11,0,2,4,118,2,0,0,1,0,6,88,3,Daniel Krutzen
1,Forge,Daniel Krutzen,90,0.07,0.00,0,1,5,103,2,0,2,4,2,7,56,4,Daniel Krutzen
2,Forge,Daniel Krutzen,90,0.02,0.14,0,1,4,96,3,2,1,3,0,5,49,3,Daniel Krutzen
3,Forge,Daniel Krutzen,90,0.00,0.08,0,1,2,95,0,0,2,5,3,3,53,2,Daniel Krutzen
4,Forge,Daniel Krutzen,90,0.07,0.07,0,2,3,94,1,1,1,4,2,7,56,3,Daniel Krutzen


In [24]:
forwards_line = pd.DataFrame()
for i in line_chart_dict['fc']:
    forwards_line = pd.concat([forwards_line,i])
forwards_line = add_display_name(forwards_line)
forwards_line.head()

,team,name,Min,ExpG,ExpA,Goal,Off,PsOpHfFl,PsAtt,PsOpHfScs,...,Touches,DuelLs,Recovery,FlComA3,FlSufM3,Disposs,TchsA3,TouchOpBox,PsCmpA3,display
0,Cavalry,Malyk Hamilton,90,0.10,0.06,0,1,5,49,19,...,80,4,4,0,0,1,20,3,7,Malyk Hamilton
1,Cavalry,Malyk Hamilton,90,0.00,0.00,0,0,2,52,17,...,77,4,6,1,0,0,12,0,8,Malyk Hamilton
2,Cavalry,Malyk Hamilton,90,0.03,0.00,0,1,4,42,12,...,74,3,9,0,0,2,20,2,7,Malyk Hamilton
3,Cavalry,Malyk Hamilton,90,0.00,0.02,0,0,3,51,15,...,70,6,9,0,0,0,9,0,4,Malyk Hamilton
4,Cavalry,Malyk Hamilton,90,0.00,0.00,0,0,1,46,23,...,69,1,5,0,0,0,21,1,13,Malyk Hamilton


In [25]:
midfielders_line = pd.DataFrame()
for i in line_chart_dict['mc']:
    midfielders_line = pd.concat([midfielders_line,i])
midfielders_line = add_display_name(midfielders_line)
midfielders_line.head()

,team,name,Min,ExpG,ExpA,Goal,Off,PsOpHfFl,PsAtt,PsOnHfScs,...,SucflDuels,Touches,DuelLs,DefTouch,Recovery,TchsA3,TchsM3,PsCmpM3,PsCmpA3,display
0,Pacific,Alexander Gonzalez Moreno,90,0.00,0.00,0,0,1,104,68,...,6,114,1,2,9,6,75,64,2,Alexander González
1,Pacific,Alexander Gonzalez Moreno,90,0.00,0.00,0,0,10,89,42,...,9,106,9,3,17,11,57,38,7,Alexander González
2,Pacific,Alexander Gonzalez Moreno,90,0.03,0.00,0,0,9,74,38,...,16,97,7,11,8,8,63,42,3,Alexander González
3,Pacific,Alexander Gonzalez Moreno,90,0.02,0.48,0,1,5,84,36,...,4,94,4,3,8,5,66,52,3,Alexander González
4,Pacific,Alexander Gonzalez Moreno,85,0.00,0.13,0,0,9,85,52,...,4,92,3,2,14,3,56,42,1,Alexander González


In [26]:
keepers_line = pd.DataFrame()
for i in line_chart_dict['gc']:
    keepers_line = pd.concat([keepers_line,i])
keepers_line = add_display_name(keepers_line)
keepers_line.head()

,team,name,Min,CleanSheet,PsOnHfFl,Recovery,GoalCncd,GoalCnIBx,Saves,SvClct,SvDive,SvHands,SvInBox,SvOutBox,SvPrdSaf,SvStand,BgChncFace,ExpGAg,display
0,Pacific,Nolan Wirth,90,0,7,1,1,1,3,1,2,3,1,2,2,1,1,1.20,Nolan Wirth
1,Pacific,Nolan Wirth,90,0,2,3,1,1,5,3,1,5,2,3,2,2,1,1.34,Nolan Wirth
2,Pacific,Nolan Wirth,90,0,4,2,3,3,7,1,6,6,3,4,4,0,4,2.57,Nolan Wirth
3,Pacific,Nolan Wirth,90,0,3,1,1,1,7,2,3,6,4,3,2,2,8,3.45,Nolan Wirth
4,Pacific,Nolan Wirth,90,0,4,4,2,1,5,4,1,5,3,2,1,4,0,0.76,Nolan Wirth


In [27]:
def percentage_check(x):
    if '%' in x:
        return float(re.sub('%','',x))
    else:
        return x

In [28]:
for col in team_game.columns:
    if team_game[col].dtypes == 'object':
        team_game[col] = team_game[col].apply(lambda x: percentage_check(x))
        #print(col,team_game[col].dtypes)
        
for col in team_totals.columns:
    if team_totals[col].dtypes == 'object':
        team_totals[col] = team_totals[col].apply(lambda x: percentage_check(x))
        #print(col,team_game[col].dtypes)

In [29]:
player_totals['pos'] = player_totals['Position'].apply(lambda x: ''.join(s[0] for s in x.lower().split() if s != '-'))

In [30]:
def get_position(x):
    try:
        return player_info[player_info['name'] == x]['position']
    except:
        return player_info[player_info['display'] == x]['position']

In [31]:
#player_totals['pos_info'] = player_totals['name'].apply(lambda x: get_position(x))

In [32]:
missing, position = [], []
for name in player_totals[['firstName','lastName']].values:
    fullname = name[0]+' '+name[1]
    try:
        if player_info[player_info['name'] == fullname].values[0][1]:
            try:
                position.append(player_info[player_info['name'] == fullname]['position'].values[0])
                pass
            except:
                position.append(player_info[player_info['display'] == fullname]['position'].values[0])
                pass
    except:
        try:
            if player_info[player_info['display'] == name[0]+' '+name[1]].values[0][1]:
                try:
                    position.append(player_info[player_info['display'] == fullname]['position'].values[0])
                    pass
                except:
                    position.append(player_info[player_info['name'] == fullname]['position'].values[0])
                    pass
                
        except:
            missing.append(name[0]+' '+name[1])
            position.append('999')
            print('MISSING',name[0]+' '+name[1])

MISSING Tyrone Venhola
MISSING Baj Maan
MISSING Chris Ezoua
MISSING Svyatik Artemenko
MISSING Luke Iacobellis
MISSING Jordan Santiago
MISSING Sachdev Grewal


In [33]:
player_totals.shape[0],len(position)

(172, 172)

In [34]:
player_totals['pos_info'] = position

In [35]:
player_totals['name'] = [x[0]+' '+x[1] for x in player_totals[['firstName','lastName']].values]
player_totals['TcklMade%'] = [float(x[:-1]) if x != '-' else 0.0 for x in player_totals['TcklMade%']]

In [36]:
skip_cols = ['Rank','GM','playerId','optaPersonId','teamId','optaTeamId','leagueId','league','season','seasonId','scatterId','scatterExtra','gameId','oppOptaTeamId']

In [37]:
cols = [x for x in player_totals.columns if x not in skip_cols]

In [38]:
rated_goalscorers = player_totals[['name','Goal','Min']].groupby('name').sum().sort_values(by='Goal',ascending=False).copy()
rated_goalscorers = rated_goalscorers.reset_index()
rated_goalscorers.head(10)

,name,Goal,Min
0,Tristan Daniel Borges,13,1973
1,Terran Campbell,11,2334
2,Dominique Malonga,11,1961
3,Easton Ongaro,10,1167
4,Rodrigo Pablo Gattas Bertoni,9,1958
5,Ryan Telfer,8,2216
6,Akeem Garcia,7,2082
7,Marco Bustos,7,2249
8,Oliver Minatel,7,1467
9,Simon Karlsson Adjei,7,1422


In [39]:
rated_assists = player_totals[['name','Ast','Min']].groupby('name').sum().sort_values(by='Ast',ascending=False).copy()
rated_assists = rated_assists.reset_index()
rated_assists.head(10)

,name,Ast,Min
0,Tristan Daniel Borges,5,1973
1,Michael Petrasso,5,1301
2,Blake Smith,5,2004
3,Ben Fisk,5,1663
4,Kwame Awuah,5,1974
5,Kyle Edward Bekker,5,2443
6,José Escalante,4,1613
7,Julian Büscher,4,1768
8,Diyaeddine Abzi,4,1814
9,Elijah Adekugbe,4,1575


position_dict = {'d':['Right Centre Back','Left Centre Back','Right Back','Central Defender','Right Winger','Left Back','Left Winger','Sub - Defender'],
                 'dc':['team','Age','pos','name','Min','Goal','GoalDirFK','GoalHead','Clrnce','Int','SucflDuels','DuelLs','TcklMade%'],
                 'f':['Right Winger', 'Left Winger','Centre Forward', 'Second Striker', 'Sub - Attacker'],
                 'fc':['team','Age','pos','name','Min','ExpG','ExpA','Goal','PenGoal','GoalDirFK','GoalHead','Ast','PsAtt','PsOnHfScs','PsOpHfScs','BgChnc','BgChncCrtd','ChncOpnPl','Chance','SOG','SOTInBox','SOTOBox','ShotDirFK','SOTHead'],
                 'm':['Central Midfielder','Defensive Midfielder','Right Attacking Midfielder','Left Midfielder','Right Midfielder','Centre Attacking Midfielder','Left Attacking Midfielder','Sub - Midfielder'],
                 'mc':['team','Age','pos','name','Min','ExpG','ExpA','Goal','PenGoal','GoalDirFK','GoalHead','Ast','PsAtt','PsOnHfScs','PsOpHfScs','Crosses','Touches','DuelLs','SucflDuels','TcklMade%'],
                 'k':['Goalkeeper','Sub - Goalkeeper'],
                 'kc':['team','Age','pos','name','Min','CleanSheet','CrossPunch','Saves','SvInBox','SvOutBox','SvPrdDng','SvPrdSaf','SvReach','SvStoop','KeeperCtch','GoalCncd']}
                 
### NEWER DICT

position_dict = {'d':['Right Centre Back','Left Centre Back','Right Back','Central Defender','Right Winger','Left Back','Left Winger','Sub - Defender'],
                 'dc':['team','Age','pos','name','Min','Goal','PsOpHfFl','PsOnHfFl','PsAtt','PsOnHfScs','SucflDuels','Touches','DuelLs','SucflTkls','Int','DefTouch','Clrnce','Recovery','TchsD3','TchsM3','PsAttD3','PsAttM3','PsDist','Aerials'],
                 'f':['Right Winger', 'Left Winger','Centre Forward', 'Second Striker', 'Sub - Attacker'],
                 'fc':['team','Age','pos','name','Min','ExpG','ExpA','Goal','SubOn','PsOpHfFl','PsAtt','PsOpHfScs','ChncOpnPl','SucflDuels','Touches','DuelLs','Recovery','FlComA3','FlComM3','FlSufM3','BgChncFace','Disposs','TchsA3','TchsM3','TouchOpBox','PsCmpA3','PsDist'],
                 'm':['Central Midfielder','Defensive Midfielder','Right Attacking Midfielder','Left Midfielder','Right Midfielder','Centre Attacking Midfielder','Left Attacking Midfielder','Sub - Midfielder'],
                 'mc':['team','Age','pos','name','Min','ExpG','ExpA','Goal','Off','ShotRF','PsOpHfFl','PsOnHfFl','PsAtt','PsOnHfScs','PsOpHfScs','SucflDuels','Touches','DuelLs','DefTouch','Recovery','BgChncFace','TchsA3','TchsD3','TchsM3','PsCmpM3','PsCmpA3','PsAttD3','PsAttM3','PsDist'],
                 'k':['Goalkeeper','Sub - Goalkeeper'],
                 'kc':['team','Age','pos','name','Min','CleanSheet','PsOpHfFl','PsOnHfFl','PsAtt','PsOnHfScs','PsOpHfScs','Touches','Recovery','GoalCncd','GoalCnIBx','Saves','SvClct','SvDive','SvHands','SvInBox','SvOutBox','SvPrdSaf','SvStand','BgChncFace','TchsD3','PsCmpD3','PsAttD3','PsDist','ExpGAg']}

In [40]:
position_dict = {'d':['Right Centre Back','Left Centre Back','Right Back','Central Defender','Right Winger','Left Back','Left Winger','Sub - Defender'],
                 'dc':['team','name','Min','ExpG','ExpA','Goal','PsOnHfFl','SucflDuels','Touches','DuelLs','SucflTkls','Int','DefTouch','Clrnce','Recovery','TchsM3','Aerials'],
                 'f':['Right Winger', 'Left Winger','Centre Forward', 'Second Striker', 'Sub - Attacker'],
                 'fc':['team','name','Min','ExpG','ExpA','Goal','Off','PsOpHfFl','PsAtt','PsOpHfScs','ChncOpnPl','SucflDuels','Touches','DuelLs','Recovery','FlComA3','FlSufM3','Disposs','TchsA3','TouchOpBox','PsCmpA3'],
                 'm':['Central Midfielder','Defensive Midfielder','Right Attacking Midfielder','Left Midfielder','Right Midfielder','Centre Attacking Midfielder','Left Attacking Midfielder','Sub - Midfielder'],
                 'mc':['team','name','Min','ExpG','ExpA','Goal','Off','PsOpHfFl','PsAtt','PsOnHfScs','PsOpHfScs','SucflDuels','Touches','DuelLs','DefTouch','Recovery','TchsA3','TchsM3','PsCmpM3','PsCmpA3'],
                 'k':['Goalkeeper','Sub - Goalkeeper'],
                 'kc':['team','name','Min','CleanSheet','PsOnHfFl','Recovery','GoalCncd','GoalCnIBx','Saves','SvClct','SvDive','SvHands','SvInBox','SvOutBox','SvPrdSaf','SvStand','BgChncFace','ExpGAg']}

In [41]:
def sort_player_stats(player_list):
    stat_columns = [x for x in player_list.columns[4:]]
                
    player_stats_df = player_list.copy()
    player_stats_df = player_stats_df.fillna(0.0)
    
    try:
        if 'PsAtt' in stat_columns:
            player_stats_df['Pass%'] = round(round((player_stats_df['PsOnHfScs'] + player_stats_df['PsOpHfScs']) / player_stats_df['PsAtt'],2)*100,2)
            stat_columns.append('Pass%')
            for col in ['PsAtt','PsOnHfScs','PsOpHfScs']:
                player_stats_df.pop(col)
                stat_columns.remove(col)

        if 'SucflDuels' in stat_columns:
            player_stats_df['DuelsW%'] = round(round(player_stats_df['SucflDuels'] / (player_stats_df['SucflDuels'] + player_stats_df['DuelLs']),2)*100,2)
            stat_columns.append('DuelsW%')
            for col in ['SucflDuels','DuelLs']:
                player_stats_df.pop(col)
                stat_columns.remove(col)
    except:
        print('Failed to find columns')
        pass
    
    eval_ = player_stats_df.describe().T
    overall = []
    for i in range(player_stats_df.shape[0]):
        a = []
        for col in stat_columns:
            if (col == 'ExpG') | ((col == 'ExpA')):
                pass
            else:
                try:
                    if eval_['max'][col] == 0:
                        result = round(player_stats_df.iloc[i][col] / 1,2)
                    else:
                        result = round(player_stats_df.iloc[i][col] / eval_['max'][col],2)
                except Exception as e:
                    print(f"Error encountered in {col} getting overall for player: {player_stats_df.iloc[i]['name']}")
                    print(e)
                a.append(result) # append the result into the list
        overall.append(round(sum(a) / len(a),2))
    player_stats_df.insert(4,'overall',overall)
    
    player_stats_df = player_stats_df.sort_values(by='overall', ascending=False)
    display,number = [],[]
    
    for i in range(player_stats_df.shape[0]):
        name = player_stats_df.iloc[i]['name']
        try:
            display.append(player_info[player_info['name'] == name]['display'].values[0])
            number.append(player_info[player_info['name'] == name]['number'].values[0])
        except:
            display.append(player_info[player_info['display'] == name]['display'].values[0])
            number.append(player_info[player_info['display'] == name]['number'].values[0])
    player_stats_df['display'] = display
    player_stats_df['number'] = number
    return player_stats_df

In [42]:
'''try:
    defenders = player_totals[player_totals['Position'].isin(position_dict['d'])][cols]
    defenderslst = defenders[position_dict['dc']].copy()
    forwards = player_totals[player_totals['Position'].isin(position_dict['f'])][cols]
    forwardslst = forwards[position_dict['fc']].copy()
    midfielders = player_totals[player_totals['Position'].isin(position_dict['m'])][cols]
    midfielderslst = midfielders[position_dict['mc']].copy()
    keepers = player_totals[player_totals['Position'].isin(position_dict['k'])][cols]
    keeperslst = keepers[position_dict['kc']].copy()
except:
    print('Error getting Stats')'''

"try:\n    defenders = player_totals[player_totals['Position'].isin(position_dict['d'])][cols]\n    defenderslst = defenders[position_dict['dc']].copy()\n    forwards = player_totals[player_totals['Position'].isin(position_dict['f'])][cols]\n    forwardslst = forwards[position_dict['fc']].copy()\n    midfielders = player_totals[player_totals['Position'].isin(position_dict['m'])][cols]\n    midfielderslst = midfielders[position_dict['mc']].copy()\n    keepers = player_totals[player_totals['Position'].isin(position_dict['k'])][cols]\n    keeperslst = keepers[position_dict['kc']].copy()\nexcept:\n    print('Error getting Stats')"

In [43]:
try:
    defenders = player_totals[player_totals['pos_info'] == 'd'][cols]
    defenderslst = defenders[position_dict['dc']].copy()
    forwards = player_totals[player_totals['pos_info'] == 'f'][cols]
    forwardslst = forwards[position_dict['fc']].copy()
    midfielders = player_totals[player_totals['pos_info'] == 'm'][cols]
    midfielderslst = midfielders[position_dict['mc']].copy()
    keepers = player_totals[player_totals['pos_info'] == 'g'][cols]
    keeperslst = keepers[position_dict['kc']].copy()
except:
    print('Error getting Stats')

In [44]:
def get_team_short(x):
    return team_names[x]

In [45]:
try:
    defenderslst = sort_player_stats(defenderslst)
    defenderslst['team'] = defenderslst['team'].apply(lambda x: get_team_short(x))
    print(defenderslst.head())
except:
    print('Error getting Stats')

   team             name   Min  ExpG  overall  ExpA  Goal  PsOnHfFl  Touches  \
1   cav   Dominick Zator  2425  3.09     0.82  3.08     1       126     1933   
4   yor  Luca Gasparotto  2520  1.54     0.82  1.50     1        96     1729   
7   fce     Mélé Temguia  2429  2.41     0.80  0.60     2        98     1680   
0   for   Daniel Krutzen  2470  1.32     0.79  1.56     1        87     2029   
35  hfx   Matthew Arnone  2094  2.10     0.72  0.22     2        86     1181   

    SucflTkls  Int  DefTouch  Clrnce  Recovery  TchsM3  Aerials  DuelsW%  \
1          33   51       174      76       142     780      149     63.0   
4          24   49       200     117       137     593      162     72.0   
7          41   51       197      82       147     589       69     57.0   
0          24   46       197      95       154    1005       85     59.0   
35         20   49       179      99       112     407       91     62.0   

            display  number  
1    Dominick Zator       4  
4 

In [46]:
try:
    forwardslst = sort_player_stats(forwardslst)
    forwardslst['team'] = forwardslst['team'].apply(lambda x: get_team_short(x))
    print(forwardslst.head())
except:
    print('Error getting Stats')

Failed to find columns
   team                          name   Min  ExpG  overall  ExpA  Goal  Off  \
36  cav                Nico Pasquotti  1943  5.77     0.76  3.60     5   13   
56  pac               Terran Campbell  2334  8.89     0.76  2.86    11   24   
55  fce                  Oumar Diouck  1976  4.91     0.73  3.48     6   17   
27  hfx              Mohamed Kourouma  1988  4.20     0.72  6.82     1   32   
50  yor  Rodrigo Pablo Gattas Bertoni  1958  9.19     0.69  1.99     9   25   

    PsOpHfFl  PsAtt  ...  DuelLs  Recovery  FlComA3  FlSufM3  Disposs  TchsA3  \
36       112    477  ...     212       114       25       22       37     660   
56       114    569  ...     206        81       15       11       52     439   
55       114    583  ...     156       105       28       28       30     421   
27       161    702  ...     141       169        3       19       37     577   
50       135    728  ...     119        87        9       13       23     412   

    TouchOpBox 

In [47]:
try:
    midfielderslst = sort_player_stats(midfielderslst)
    midfielderslst['team'] = midfielderslst['team'].apply(lambda x: get_team_short(x))
    print(midfielderslst.head())
except:
    print('Error getting Stats')

   team                   name   Min  ExpG  overall  ExpA  Goal  Off  \
3   for     Kyle Edward Bekker  2443  5.83     0.80  5.29     4   27   
2   yor        Manuel Aparicio  2159  2.30     0.77  2.20     2   21   
12  cav         Julian Büscher  1768  2.84     0.72  6.54     5   14   
11  yor            Ryan Telfer  2216  7.15     0.72  5.59     8   21   
22  for  Tristan Daniel Borges  1973  9.34     0.70  4.10    13   10   

    PsOpHfFl  Touches  DefTouch  Recovery  TchsA3  TchsM3  PsCmpM3  PsCmpA3  \
3        199     1783        62       162     590     971      721      296   
2        232     1788        98       160     492    1047      668      197   
12       216     1518       109       187     479     807      504      164   
11       161     1533        83       130     698     654      388      179   
22       136     1353        88       145     712     524      252      237   

    Pass%  DuelsW%         display  number  
3    82.0     35.0     Kyle Bekker       3  
2 

In [48]:
try:
    keeperslst = sort_player_stats(keeperslst)
    keeperslst['team'] = keeperslst['team'].apply(lambda x: get_team_short(x))
    print(keeperslst.head())
except:
    print('Error getting Stats')

   team            name   Min  CleanSheet  overall  PsOnHfFl  Recovery  \
38  yor   Nathan Ingham  2340           6     0.94        53        38   
60  fce    Connor James  1979           7     0.78        68        21   
59  cav  Marco Carducci  2250           9     0.72        49        32   
98  pac     Nolan Wirth  1080           1     0.70        42        17   
86  val    Tyson Farago  1440           2     0.65        30        18   

    GoalCncd  GoalCnIBx  Saves  ...  SvDive  SvHands  SvInBox  SvOutBox  \
38        34         25     88  ...      52       70       60        28   
60        24         21     67  ...      29       57       38        29   
59        17         17     65  ...      31       61       33        32   
98        24         22     59  ...      26       51       32        27   
86        30         26     47  ...      14       38       31        16   

    SvPrdSaf  SvStand  BgChncFace  ExpGAg         display number  
38        37       14          44   2

In [49]:
disciplinelst = player_totals[['team','name','Min','Red','2ndYellow','Yellow','FlComA3','FlComD3','FlComM3']].copy()
disciplinelst['Fls'] = disciplinelst['FlComA3'] + disciplinelst['FlComD3'] + disciplinelst['FlComM3']
for col in ['FlComA3','FlComD3','FlComM3']:
    disciplinelst.pop(col)
disciplinelst = disciplinelst.sort_values(by='Red',ascending=False)
disciplinelst['display'] = '-'
for i in range(disciplinelst.shape[0]):
    name = disciplinelst.at[i,'name']
    try:
        disciplinelst.at[i,'display'] = player_info[player_info['name'] == name]['display'].values[0]
    except:
        disciplinelst.at[i,'display'] = name
disciplinelst['team'] = disciplinelst['team'].apply(lambda x: ccd_names[x])
disciplinelst['short'] = disciplinelst['team'].apply(lambda x: team_names[x])
disciplinelst.head()

,team,name,Min,Red,2ndYellow,Yellow,Fls,display,short
109,Cavalry FC,Mauro Eustaquio,533,1,0,2,16,Mauro Eustaquio,cav
14,Pacific FC,Alexander Gonzalez Moreno,1746,1,0,4,34,Alexander González,pac
24,FC Edmonton,Jeannot Esua,1798,1,0,2,20,Jeannot Esua,fce
22,Forge FC,Tristan Daniel Borges,1973,1,0,2,38,Tristan Borges,for
62,Cavalry FC,José Escalante,1613,1,0,3,24,José Escalante,cav


In [50]:
def norm(x,data):
    m = data.max()
    if m == 0:
        m = 1
    x = (x / 90)
    return round(x,2)
    
def get_column_overall(lst):
    data = stats[lst]
    data = data.groupby(['team']).sum()
    data['overall'] = data.sum(axis=1) / data.shape[1]
    data['overall'] = data['overall'] / data['overall'].max()
    data['overall'] = data['overall'] - 0.1
    data = data[['overall']]
    data = data.reset_index()
    data.pop('team')
    return data['overall']

def get_90(data):
    df = data.copy()
    cols = [x for x in data.columns if x not in ['team','Age','pos','name','Min','overall','display']]
    df['Min'] = (round(df['Min'] / 90)).astype('int')
    for col in cols:
        df[col] = round(df[col]/df['Min'],2)#.apply(lambda x: norm(x,df[col]))
    df['Min'] = 90
    #df['team'] = df['team'].apply(lambda x: ccd_names[x])
    return df

In [51]:
forwardslst.columns

Index(['team', 'name', 'Min', 'ExpG', 'overall', 'ExpA', 'Goal', 'Off',
       'PsOpHfFl', 'PsAtt', 'PsOpHfScs', 'ChncOpnPl', 'SucflDuels', 'Touches',
       'DuelLs', 'Recovery', 'FlComA3', 'FlSufM3', 'Disposs', 'TchsA3',
       'TouchOpBox', 'PsCmpA3', 'display', 'number'],
      dtype='object')

In [52]:
midfielderslst.columns

Index(['team', 'name', 'Min', 'ExpG', 'overall', 'ExpA', 'Goal', 'Off',
       'PsOpHfFl', 'Touches', 'DefTouch', 'Recovery', 'TchsA3', 'TchsM3',
       'PsCmpM3', 'PsCmpA3', 'Pass%', 'DuelsW%', 'display', 'number'],
      dtype='object')

In [53]:
defenderslst.columns

Index(['team', 'name', 'Min', 'ExpG', 'overall', 'ExpA', 'Goal', 'PsOnHfFl',
       'Touches', 'SucflTkls', 'Int', 'DefTouch', 'Clrnce', 'Recovery',
       'TchsM3', 'Aerials', 'DuelsW%', 'display', 'number'],
      dtype='object')

In [55]:
keeperslst.columns

Index(['team', 'name', 'Min', 'CleanSheet', 'overall', 'PsOnHfFl', 'Recovery',
       'GoalCncd', 'GoalCnIBx', 'Saves', 'SvClct', 'SvDive', 'SvHands',
       'SvInBox', 'SvOutBox', 'SvPrdSaf', 'SvStand', 'BgChncFace', 'ExpGAg',
       'display', 'number'],
      dtype='object')

In [56]:
some_list = ['abc-123', 'def-456', 'ghi-789', 'abc-456']
[s for s in some_list if "abc" in s][0]

'abc-123'

In [57]:
ccd_names['Valour']

'Valour FC'

In [58]:
try:
    defenders90lst = get_90(defenderslst)
    forwards90lst = get_90(forwardslst)
    midfielders90lst = get_90(midfielderslst)
    keepers90lst = get_90(keeperslst)
except:
    print('Error getting Stats')

In [59]:
forwards90lst.head()

,team,name,Min,ExpG,overall,ExpA,Goal,Off,PsOpHfFl,PsAtt,...,DuelLs,Recovery,FlComA3,FlSufM3,Disposs,TchsA3,TouchOpBox,PsCmpA3,display,number
36,cav,Nico Pasquotti,90,0.26,0.76,0.16,0.23,0.59,5.09,21.68,...,9.64,5.18,1.14,1.00,1.68,30.00,4.50,6.86,Nico Pasquotti,0.77
56,pac,Terran Campbell,90,0.34,0.76,0.11,0.42,0.92,4.38,21.88,...,7.92,3.12,0.58,0.42,2.00,16.88,2.96,6.54,Terran Campbell,0.54
55,fce,Oumar Diouck,90,0.22,0.73,0.16,0.27,0.77,5.18,26.50,...,7.09,4.77,1.27,1.27,1.36,19.14,3.32,7.05,Oumar Diouck,2.05
27,hfx,Mohamed Kourouma,90,0.19,0.72,0.31,0.05,1.45,7.32,31.91,...,6.41,7.68,0.14,0.86,1.68,26.23,2.45,7.23,Mohamed Kourouma,0.55
50,yor,Rodrigo Pablo Gattas Bertoni,90,0.42,0.69,0.09,0.41,1.14,6.14,33.09,...,5.41,3.95,0.41,0.59,1.05,18.73,3.91,7.14,Rodrigo Gattas,1.00


In [60]:
def str_check(x):
    try:
        return [s for s in team_name_list if x in s][0]
    except:
        print(x)

In [61]:
### Get Best Eleven

In [62]:
best_eleven = cpl.get_best_eleven(team_ref,forwardslst,midfielderslst,defenderslst,keeperslst,player_info)

In [63]:
best_eleven

,image,name,number,flag,overall,position,link,colour
0,Nathan-Ingham.jpg,Nathan Ingham,1,Canada.png,0.94,g,https://en.wikipedia.org/wiki/Nathan_Ingham,cpl-y9-old
1,Dominick-Zator.jpg,Dominick Zator,4,Canada.png,0.82,d,https://en.wikipedia.org/wiki/Dominick_Zator,cpl-cfc
2,Luca-Gasparotto.jpg,Luca Gasparotto,13,Canada.png,0.82,d,https://en.wikipedia.org/wiki/Luca_Gasparotto,cpl-y9-old
3,Mélé-Temguia.jpg,Mélé Temguia,8,Germany.png,0.80,d,https://en.wikipedia.org/wiki/Mélé_Temguia,cpl-fce
4,Kyle-Bekker.jpg,Kyle Edward Bekker,3,Canada.png,0.80,m,https://en.wikipedia.org/wiki/Kyle_Bekker,cpl-ffc
5,Manuel-Aparicio.jpg,Manuel Aparicio,10,Canada.png,0.77,m,https://en.wikipedia.org/wiki/Manuel_Aparicio,cpl-y9-old
6,Julian-Buscher.jpg,Julian Büscher,8,Germany.png,0.72,m,https://en.wikipedia.org/wiki/Julian_Büscher,cpl-cfc
7,Ryan-Telfer.jpg,Ryan Telfer,18,Canada.png,0.72,m,https://en.wikipedia.org/wiki/Ryan_Telfer,cpl-y9-old
8,Tristan-Borges.jpg,Tristan Daniel Borges,4,Canada.png,0.70,m,https://en.wikipedia.org/wiki/Tristan_Borges,cpl-ffc
9,Nico-Pasquotti.jpg,Nico Pasquotti,17,Canada.png,0.76,f,https://en.wikipedia.org/wiki/Nico_Pasquotti,cpl-cfc


### Get TEAM STATS

In [64]:
team_game['points'] = 0
team_game['win'] = 0
team_game['draw'] = 0
team_game['loss'] = 0

In [65]:
try:
    if year == '2019':
        team_totals['finish'] = round((team_totals['GoalDirFK']+team_totals['GoalInBx']+team_totals['GoalOBx']+team_totals['GoalOP']+team_totals['BgChncCrtd'])/7,2)
        team_totals['def'] = round((team_totals['SucflTkls']+team_totals['Clrnce']+team_totals['Int']+team_totals['SucflDuels'])/4,2)
        team_totals['mid'] = round((team_totals['PsOpHfScs']+team_totals['PsCmpM3']+team_totals['GoalOP']+team_totals['GoalOBx']+team_totals['ThrghBlCmp']+team_totals['Recovery']+team_totals['%PassFwd'])/7,2)
        team_totals['att'] = round((team_totals['TchsA3']+team_totals['PsCmpA3']+team_totals['SOG']+team_totals['SOTInBox']+team_totals['SOTOBox']+team_totals['Sc1v1InBox'])/6,2)
    else:
        team_totals['finish'] = round((team_totals['GoalDirFK']+team_totals['GoalInBx']+team_totals['GoalOBx']+team_totals['GoalOP']+team_totals['BgChncCrtd'])/7,2)
        team_totals['def'] = round((team_totals['SucflTkls']+team_totals['Clrnce']+team_totals['Int']+team_totals['SucflDuels']+team_totals['Defensive Blocks'])/5,2)
        team_totals['mid'] = round((team_totals['TchsM3']+team_totals['PsOpHfScs']+team_totals['PsCmpM3']+team_totals['GoalOP']+team_totals['GoalOBx']+team_totals['ThrghBlCmp']+team_totals['Recovery']+team_totals['%PassFwd'])/8,2)
        team_totals['att'] = round((team_totals['TchsA3']+team_totals['PsCmpA3']+team_totals['CrossOpen']+team_totals['Crosses']+team_totals['SOG']+team_totals['SOTInBox']+team_totals['SOTOBox']+team_totals['Sc1v1InBox'])/8,2)

    team_totals['net'] = round((team_totals['Saves']+team_totals['SvClct']+team_totals['SvDive']+team_totals['SvPrdSaf']+team_totals['SvReach']+team_totals['SvInBox']+team_totals['SvOutBox'])/7,2)
except:
    pass
'''if year == '2019':
    team_game['finish'] = round((team_game['GoalDirFK']+team_game['GoalInBx']+team_game['GoalOBx']+team_game['GoalOP']+team_game['BgChncCrtd'])/7,2)
    team_game['def'] = round((team_game['SucflTkls']+team_game['Clrnce']+team_game['Int']+team_game['SucflDuels'])/4,2)
    team_game['mid'] = round((team_game['PsOpHfScs']+team_game['PsCmpM3']+team_game['GoalOP']+team_game['GoalOBx']+team_game['ThrghBlCmp']+team_game['Recovery']+team_game['%PassFwd'])/7,2)
    team_game['att'] = round((team_game['TchsA3']+team_game['PsCmpA3']+team_game['SOG']+team_game['SOTInBox']+team_game['SOTOBox']+team_game['Sc1v1InBox'])/6,2)
else:
    team_game['finish'] = round((team_game['GoalDirFK']+team_game['GoalInBx']+team_game['GoalOBx']+team_game['GoalOP']+team_game['BgChncCrtd'])/7,2)
    team_game['def'] = round((team_game['SucflTkls']+team_game['Clrnce']+team_game['Int']+team_game['SucflDuels']+team_game['Defensive Blocks'])/5,2)
    team_game['mid'] = round((team_game['TchsM3']+team_game['PsOpHfScs']+team_game['PsCmpM3']+team_game['GoalOP']+team_game['GoalOBx']+team_game['ThrghBlCmp']+team_game['Recovery']+team_game['%PassFwd'])/8,2)
    team_game['att'] = round((team_game['TchsA3']+team_game['PsCmpA3']+team_game['CrossOpen']+team_game['Crosses']+team_game['SOG']+team_game['SOTInBox']+team_game['SOTOBox']+team_game['Sc1v1InBox'])/8,2)

team_game['net'] = round((team_game['Saves']+team_game['SvClct']+team_game['SvDive']+team_game['SvPrdSaf']+team_game['SvReach']+team_game['SvInBox']+team_game['SvOutBox'])/7,2)'''

"if year == '2019':\n    team_game['finish'] = round((team_game['GoalDirFK']+team_game['GoalInBx']+team_game['GoalOBx']+team_game['GoalOP']+team_game['BgChncCrtd'])/7,2)\n    team_game['def'] = round((team_game['SucflTkls']+team_game['Clrnce']+team_game['Int']+team_game['SucflDuels'])/4,2)\n    team_game['mid'] = round((team_game['PsOpHfScs']+team_game['PsCmpM3']+team_game['GoalOP']+team_game['GoalOBx']+team_game['ThrghBlCmp']+team_game['Recovery']+team_game['%PassFwd'])/7,2)\n    team_game['att'] = round((team_game['TchsA3']+team_game['PsCmpA3']+team_game['SOG']+team_game['SOTInBox']+team_game['SOTOBox']+team_game['Sc1v1InBox'])/6,2)\nelse:\n    team_game['finish'] = round((team_game['GoalDirFK']+team_game['GoalInBx']+team_game['GoalOBx']+team_game['GoalOP']+team_game['BgChncCrtd'])/7,2)\n    team_game['def'] = round((team_game['SucflTkls']+team_game['Clrnce']+team_game['Int']+team_game['SucflDuels']+team_game['Defensive Blocks'])/5,2)\n    team_game['mid'] = round((team_game['TchsM3'

In [66]:
for total in team_game['scatterExtra'].unique():
    r = team_game[team_game['scatterExtra'] == total][['team','Goal','GWG']]
    if r['GWG'].sum() > 0:
        w = r[r['GWG'] == 1].index.values.astype(int)
        l = r[r['GWG'] == 0].index.values.astype(int)
        team_game.at[w[0],'points'] = 3
        team_game.at[w[0],'win'] = 1
        team_game.at[l[0],'points'] = 0
        team_game.at[l[0],'loss'] = 1
    else:
        v = r.index.values.astype(int)
        team_game.at[v[0],'points'] = 1
        team_game.at[v[0],'draw'] = 1
        team_game.at[v[1],'points'] = 1
        team_game.at[v[1],'draw'] = 1

### TEAM RADAR

In [67]:
team_radar = team_totals[['team','finish','def','att','mid','net','BgChnc','BgChncCrtd','ChncOpnPl','Chance','CleanSheet','CrossPunch','Saves','Goal','GoalCncd']].copy()
for col in team_radar:
    if col == 'team':
        pass
    else:
        team_radar[col] = team_radar[col].apply(lambda x: round((x / team_radar[col].max())-.05,2))
team_radar['big'] = round((team_radar['BgChnc']+team_radar['ChncOpnPl']+team_radar['BgChncCrtd']+team_radar['Chance'])/4,2)
team_radar['save'] = round((team_radar['CleanSheet']+team_radar['CrossPunch']+team_radar['Saves']+(team_radar['GoalCncd']*-1))/4+.3,2)
team_radar['goals'] = team_radar['Goal'] - team_radar['GoalCncd'] + 1
team_radar['goals'] = team_radar['goals'].apply(lambda x: round((x / team_radar['goals'].max())-.05,2))

for col in ['BgChnc','BgChncCrtd','ChncOpnPl','Chance','CleanSheet','Saves','GoalCncd','CrossPunch','Goal']:
    team_radar.pop(col)
team_radar['team'] = team_radar['team'].apply(lambda x: ccd_names[x])
team_radar = team_radar.sort_values(by='team')
team_radar

,team,finish,def,att,mid,net,big,save,goals
2,Cavalry FC,0.95,0.92,0.79,0.76,0.75,0.83,0.77,0.95
4,FC Edmonton,0.43,0.84,0.63,0.76,0.77,0.48,0.56,0.51
0,Forge FC,0.89,0.93,0.95,0.95,0.73,0.80,0.70,0.84
6,HFX Wanderers FC,0.32,0.95,0.57,0.63,0.75,0.47,0.71,0.41
3,Pacific FC,0.60,0.90,0.62,0.78,0.95,0.60,0.48,0.46
5,Valour FC,0.46,0.83,0.67,0.72,0.79,0.56,0.40,0.32
1,York9 FC,0.63,0.89,0.74,0.87,0.84,0.73,0.58,0.61


In [68]:
cols = [x for x in team_game.columns if x not in skip_cols]
team_game['week'] = team_game['date'].apply(lambda x: x[5:])
team_game['gd'] = team_game['Goal'] - team_game['GoalCncd']

### Standings and Get Differences

In [69]:
months = {'01': 31,'02': 28,'03': 31,'04': 30,'05': 31,'06': 30,'07': 31,'08': 31,'09': 30,'10': 31,'11': 30,'12': 31,}

In [70]:
def get_7_days(weekday):
    d = int(weekday[-2:])
    m = months[weekday[5:7]]
    p = int(weekday[5:7])-1
    y = weekday[:4]
    if p < 10:
        p = '0'+str(p)
    else:
        p = str(p)
    if d <= 7:
        return f'{y}-{p}-{str(m+d-7)}'
    else:
        return f'{y}-{p}-{str(d-7)}'

In [71]:
#regular_season = team_game[['team','week','points','win','draw','loss','gd','finish','def','att','mid','net','GWG','Goal','GoalCncd','finish','BgChnc','BgChncCrtd','ChncOpnPl','Chance','CleanSheet','CrossPunch','Saves','Red','Yellow','2ndYellow','date']]
regular_season = team_game[['team','week','points','win','draw','loss','gd','GWG','Goal','CleanSheet','Saves','Red','Yellow','2ndYellow','date']]
try:
    regular_season = regular_season[regular_season['date'] <= season.loc[0]['end']]
    matches = regular_season['win'] + regular_season['loss'] + regular_season['draw']
    regular_season.insert(0,'matches',matches)
    regular_season.pop('date')
    regular_season['team'] = regular_season['team'].apply(lambda x: ccd_names[x])
    regular_season = regular_season.groupby('team').sum().sort_values(by='points',ascending=False)
    regular_season = regular_season.reset_index()
except:
    regular_season = pd.DataFrame()

In [72]:
regular_season_prev = team_game[['team','week','points','win','draw','loss','gd','GWG','Goal','CleanSheet','Saves','Red','Yellow','2ndYellow','date']]
try:
    regular_season_prev = regular_season_prev[regular_season_prev['date'] <= get_7_days(season.loc[0]['end'])]
    matches = regular_season_prev['win'] + regular_season_prev['loss'] + regular_season_prev['draw']
    regular_season_prev.insert(0,'matches',matches)
    regular_season_prev.pop('date')
    regular_season_prev['team'] = regular_season_prev['team'].apply(lambda x: ccd_names[x])
    regular_season_prev = regular_season_prev.groupby('team').sum().sort_values(by='points',ascending=False)
    regular_season_prev = regular_season_prev.reset_index()
except:
    regular_season_prev = pd.DataFrame()

In [73]:
regular_season

,team,matches,points,win,draw,loss,gd,GWG,Goal,CleanSheet,Saves,Red,Yellow,2ndYellow
0,Cavalry FC,10,24,8,0,2,9,8,16,5,27,2,22,0
1,Forge FC,10,19,6,1,3,8,6,15,5,22,0,14,0
2,FC Edmonton,10,14,4,2,4,-1,4,8,4,29,1,11,0
3,HFX Wanderers FC,10,11,3,2,5,-3,3,8,2,27,0,18,0
4,Pacific FC,10,11,3,2,5,-4,3,11,2,25,1,13,1
5,York9 FC,10,11,2,5,3,-2,2,9,3,39,0,16,1
6,Valour FC,10,9,3,0,7,-7,3,8,2,27,0,17,0


In [74]:
regular_season_prev

,team,matches,points,win,draw,loss,gd,GWG,Goal,CleanSheet,Saves,Red,Yellow,2ndYellow
0,Cavalry FC,9,24,8,0,1,11,8,15,5,23,0,18,0
1,Forge FC,9,19,6,1,2,9,6,15,5,21,0,13,0
2,HFX Wanderers FC,9,11,3,2,4,-1,3,8,2,26,0,18,0
3,Valour FC,9,9,3,0,6,-5,3,7,2,23,0,17,0
4,FC Edmonton,8,8,2,2,4,-4,2,5,2,26,1,11,0
5,Pacific FC,9,8,2,2,5,-6,2,8,2,18,0,12,1
6,York9 FC,9,8,1,5,3,-4,1,6,3,35,0,16,1


In [75]:
#playoffs = team_game[['team','week','points','win','draw','loss','gd','finish','def','att','mid','net','GWG','Goal','GoalCncd','BgChnc','BgChncCrtd','ChncOpnPl','Chance','CleanSheet','CrossPunch','Saves','Red','Yellow','2ndYellow','date']]
playoffs = team_game[['team','week','points','win','draw','loss','gd','GWG','Goal','CleanSheet','Saves','Red','Yellow','2ndYellow','date']]
try:
    playoffs = playoffs[(playoffs['date'] >= season.loc[1]['start']) & (playoffs['date'] <= season.loc[1]['end'])]
    matches = playoffs['win'] + playoffs['loss'] + playoffs['draw']
    playoffs.insert(0,'matches',matches)
    playoffs.pop('date')
    playoffs['team'] = playoffs['team'].apply(lambda x: ccd_names[x])
    playoffs = playoffs.groupby('team').sum().sort_values(by='points',ascending=False)
    playoffs = playoffs.reset_index()
except:
    playoffs = pd.DataFrame()

In [76]:
if year == '2019':
    playoffs_prev = team_game[['team','week','points','win','draw','loss','gd','GWG','Goal','CleanSheet','Saves','Red','Yellow','2ndYellow','date']].sort_values(by='date')
    weekending = get_7_days(season.loc[1]['end'])
    try:
        playoffs_prev = playoffs_prev[(playoffs_prev['date'] >= season.loc[1]['start']) & (playoffs_prev['date'] <= weekending)]
        matches = playoffs_prev['win'] + playoffs_prev['loss'] + playoffs_prev['draw']
        playoffs_prev.insert(0,'matches',matches)
        playoffs_prev.pop('date')
        playoffs_prev['team'] = playoffs_prev['team'].apply(lambda x: ccd_names[x])
        playoffs_prev = playoffs_prev.groupby('team').sum().sort_values(by='points',ascending=False)
        playoffs_prev = playoffs_prev.reset_index()
    except:
        playoffs_prev = pd.DataFrame()

In [77]:
playoffs

,team,matches,points,win,draw,loss,gd,GWG,Goal,CleanSheet,Saves,Red,Yellow,2ndYellow
0,Cavalry FC,18,38,11,5,2,23,11,35,7,54,1,24,1
1,Forge FC,18,37,11,4,3,11,11,30,6,58,0,29,1
2,York9 FC,18,23,7,2,9,4,7,30,3,53,0,26,0
3,Pacific FC,18,20,5,5,8,-7,5,24,3,81,0,31,2
4,Valour FC,18,19,5,4,9,-15,5,22,4,63,2,40,0
5,FC Edmonton,18,18,4,6,8,-5,4,19,4,58,1,24,0
6,HFX Wanderers FC,18,17,3,8,7,-11,3,13,6,56,1,32,0


In [78]:
if year == '2019':
    for i in range(playoffs_prev.shape[0]):
        print(playoffs_prev.loc[i].values)

['Forge FC' 10 23 7 2 1 10 7 20 2 26 0 14 0]
['Cavalry FC' 11 22 6 4 1 13 6 19 6 44 0 10 0]
['FC Edmonton' 11 14 3 5 3 1 3 12 4 35 1 14 0]
['York9 FC' 11 13 4 1 6 -1 4 16 1 32 0 15 0]
['Pacific FC' 11 12 3 3 5 -2 3 17 1 57 0 18 1]
['Valour FC' 12 12 3 3 6 -10 3 14 3 37 2 27 0]
['HFX Wanderers FC' 10 8 2 2 6 -11 2 5 4 31 0 21 0]


In [79]:
#championship = team_game[['team','week','points','win','draw','loss','gd','finish','def','att','mid','net','GWG','Goal','GoalCncd','BgChnc','BgChncCrtd','ChncOpnPl','Chance','CleanSheet','CrossPunch','Saves','Red','Yellow','2ndYellow','date']]
championship = team_game[['team','week','points','win','draw','loss','gd','GWG','Goal','CleanSheet','Saves','Red','Yellow','2ndYellow','date']]
try:
    championship = championship[championship['date'] >= season.loc[2]['start']]
    matches = championship['win'] + championship['loss'] + championship['draw']
    championship.insert(0,'matches',matches)
    championship.pop('date')
    championship['team'] = championship['team'].apply(lambda x: ccd_names[x])
    championship = championship.groupby('team').sum().sort_values(by='points',ascending=False)
    championship = championship.reset_index()
except:
    championship = pd.DataFrame()

In [80]:
championship

,team,matches,points,win,draw,loss,gd,GWG,Goal,CleanSheet,Saves,Red,Yellow,2ndYellow
0,Forge FC,2,6,2,0,0,2,2,2,2,3,1,4,0
1,Cavalry FC,2,0,0,0,2,-2,0,0,0,6,1,8,0


### POWER RANKINGS

In [81]:
## --- IMPROVE THIS CODE
if year != '2019':
    power = pd.concat([regular_season.sort_values(by='team'),regular_season_prev.sort_values(by='team')]).copy()
else:
    power = pd.concat([playoffs.sort_values(by='team'),playoffs_prev.sort_values(by='team')]).copy()
#power.insert(1,'change',power.index + 1)
power = power[power.columns[:7]].groupby('team').var()
power = power.reset_index()
power['rank'] = 0
for i in range(power.shape[0]):
    if year != '2019':
        power.at[i,'rank'] = regular_season[regular_season['team'] == power.iloc[i]['team']].index[0]+1
    else:
        power.at[i,'rank'] = playoffs[playoffs['team'] == power.iloc[i]['team']].index[0]+1
def alter(x):
    if x == 0.5:
        return 1
    else:
        return int(x)
for col in power.columns[1:]:
    power[col] = power[col].apply(lambda x: alter(x))
power['move'] = power['win'] - power['loss'] + power['draw']
power['move'] = power['points'] + power['move']
def norm_col(x):
    return x / power['move'].max()
power['move'] = power['move'].apply(lambda x: norm_col(x))
power['move'] =(power['rank'] * power['move']).astype('int')
power['new'] = power['rank'] - power['move'] + 1
power['change'] = power['rank'] - power['new']
#rank = [x for x in power['rank']]
change = [x for x in power['new']]
rank = [x for x in power['rank']]
# calculate the change in ranking 
check = [x for x in set([x for x in change if change.count(x) > 1])]
n = []
for x,y in zip(change,rank):
    if x >= 8:
        if x == 8:
            n.append(x-1)
        else:
            n.append(8)
    else:
        if x in check:
            if x > y:
                n.append(x)
            else:
                n.append(x-1)
        else:
            n.append(x)
power['change'] = n
for col in ['matches','points','gd','new','win','loss','draw']:
    power.pop(col)
power['move'] = power['rank'] - power['change']
power['rank'] = power['change']
power.pop('change')
power = power.sort_values(by='rank')
power

,team,rank,move
0,Cavalry FC,1,0
2,Forge FC,2,0
6,York9 FC,3,0
3,HFX Wanderers FC,5,2
4,Pacific FC,5,-1
5,Valour FC,6,-1
1,FC Edmonton,7,-1


In [82]:
rated_goalscorers = add_display_name(rated_goalscorers)
rated_goalscorers = add_position_number_team(rated_goalscorers)
rated_goalscorers

,name,Goal,Min,display,position,number,team
0,Tristan Daniel Borges,13,1973,Tristan Borges,m,4,Forge FC
1,Terran Campbell,11,2334,Terran Campbell,f,14,Pacific FC
2,Dominique Malonga,11,1961,Dominique Malonga,f,19,Cavalry FC
3,Easton Ongaro,10,1167,Easton Ongaro,f,19,FC Edmonton
4,Rodrigo Pablo Gattas Bertoni,9,1958,Rodrigo Gattas,f,22,York9 FC
...,...,...,...,...,...,...,...
167,Federico Peña,0,315,Federico Peña,f,77,Valour FC
168,Emmanuel Zambazis,0,277,Emmanuel Zambazis,m,14,York9 FC
169,Elton John,0,1435,Elton John,m,5,HFX Wanderers FC
170,Elliot Simmons,0,969,Elliot Simmons,m,8,HFX Wanderers FC


In [83]:
rated_assists = add_display_name(rated_assists)
rated_assists = add_position_number_team(rated_assists)
rated_assists

,name,Ast,Min,display,position,number,team
0,Tristan Daniel Borges,5,1973,Tristan Borges,m,4,Forge FC
1,Michael Petrasso,5,1301,Michael Petrasso,m,9,Valour FC
2,Blake Smith,5,2004,Blake Smith,d,4,Pacific FC
3,Ben Fisk,5,1663,Ben Fisk,m,21,Pacific FC
4,Kwame Awuah,5,1974,Kwame Awuah,d,2,Forge FC
...,...,...,...,...,...,...,...
167,James Marcelin,0,1698,James Marcelin,m,14,FC Edmonton
168,Jace Kotsopoulos,0,138,Jace Kotsopoulos,m,12,Forge FC
169,Issey Nakajima-Farran,0,252,Issey Nakajima-Farran,f,11,Pacific FC
170,Hendrik Starostzik,0,1161,Hendrik Starostzik,d,16,Pacific FC


In [84]:
# file output
if defenderslst.empty:
    print('Empty DataFrame')
else:
    print('Saving Files to CSV')
    forwardslst.to_csv(f'datasets/{year}/playerstats/{year}-forwards.csv',index=False)
    midfielderslst.to_csv(f'datasets/{year}/playerstats/{year}-midfielders.csv',index=False)
    defenderslst.to_csv(f'datasets/{year}/playerstats/{year}-defenders.csv',index=False)
    keeperslst.to_csv(f'datasets/{year}/playerstats/{year}-keepers.csv',index=False)
    forwards90lst.to_csv(f'datasets/{year}/playerstats/{year}-forwards90.csv',index=False)
    midfielders90lst.to_csv(f'datasets/{year}/playerstats/{year}-midfielders90.csv',index=False)
    defenders90lst.to_csv(f'datasets/{year}/playerstats/{year}-defenders90.csv',index=False)
    keepers90lst.to_csv(f'datasets/{year}/playerstats/{year}-keepers90.csv',index=False)
    disciplinelst.to_csv(f'datasets/{year}/playerstats/{year}-discipline.csv',index=False)
    best_eleven.to_csv(f'datasets/{year}/playerstats/{year}-best_eleven.csv',index=False)
    rated_goalscorers.to_csv(f'datasets/{year}/playerstats/{year}-goalscorers.csv',index=False)
    rated_assists.to_csv(f'datasets/{year}/playerstats/{year}-assists.csv',index=False)

Saving Files to CSV


In [85]:
# file output
if defenderslst.empty:
    print('Empty DataFrame')
else:
    print('Saving Files to CSV')
    forwards_line.to_csv(f'datasets/{year}/playerstats/{year}-forwards-line.csv',index=False)
    midfielders_line.to_csv(f'datasets/{year}/playerstats/{year}-midfielders-line.csv',index=False)
    defenders_line.to_csv(f'datasets/{year}/playerstats/{year}-defenders-line.csv',index=False)
    keepers_line.to_csv(f'datasets/{year}/playerstats/{year}-keepers-line.csv',index=False)

Saving Files to CSV


In [86]:
if regular_season.empty:
    print('Empty DataFrame')
else:
    print('Saving Standings to CSV')
    regular_season.to_csv(f'datasets/{year}/league/{year}-regular-standings.csv',index=False)
    regular_season_prev.to_csv(f'datasets/{year}/league/{year}-regular-standings-prev.csv',index=False)
if playoffs.empty:
    print('Empty DataFrame')
else:
    print('Saving Playoffs to CSV')
    playoffs.to_csv(f'datasets/{year}/league/{year}-playoffs.csv',index=False)
if championship.empty:
    print('Empty DataFrame')
else:
    print('Saving Championship to CSV')
    championship.to_csv(f'datasets/{year}/league/{year}-championship.csv',index=False)

Saving Standings to CSV
Saving Playoffs to CSV
Saving Championship to CSV


In [87]:
team_radar.to_csv(f'datasets/{year}/league/{year}-radar.csv',index=False)
power.to_csv(f'datasets/{year}/league/{year}-power_rankings.csv',index=False)

# REMOVE PERCENT AND CONVERT TO NUMBERS

In [88]:
defenderslst.columns

Index(['team', 'name', 'Min', 'ExpG', 'overall', 'ExpA', 'Goal', 'PsOnHfFl',
       'Touches', 'SucflTkls', 'Int', 'DefTouch', 'Clrnce', 'Recovery',
       'TchsM3', 'Aerials', 'DuelsW%', 'display', 'number'],
      dtype='object')

In [89]:
defenders_line.columns

Index(['team', 'name', 'Min', 'ExpG', 'ExpA', 'Goal', 'PsOnHfFl', 'SucflDuels',
       'Touches', 'DuelLs', 'SucflTkls', 'Int', 'DefTouch', 'Clrnce',
       'Recovery', 'TchsM3', 'Aerials', 'display'],
      dtype='object')

In [90]:
forwards_line.columns

Index(['team', 'name', 'Min', 'ExpG', 'ExpA', 'Goal', 'Off', 'PsOpHfFl',
       'PsAtt', 'PsOpHfScs', 'ChncOpnPl', 'SucflDuels', 'Touches', 'DuelLs',
       'Recovery', 'FlComA3', 'FlSufM3', 'Disposs', 'TchsA3', 'TouchOpBox',
       'PsCmpA3', 'display'],
      dtype='object')

In [91]:
midfielders_line.columns

Index(['team', 'name', 'Min', 'ExpG', 'ExpA', 'Goal', 'Off', 'PsOpHfFl',
       'PsAtt', 'PsOnHfScs', 'PsOpHfScs', 'SucflDuels', 'Touches', 'DuelLs',
       'DefTouch', 'Recovery', 'TchsA3', 'TchsM3', 'PsCmpM3', 'PsCmpA3',
       'display'],
      dtype='object')

In [92]:
keepers_line.columns

Index(['team', 'name', 'Min', 'CleanSheet', 'PsOnHfFl', 'Recovery', 'GoalCncd',
       'GoalCnIBx', 'Saves', 'SvClct', 'SvDive', 'SvHands', 'SvInBox',
       'SvOutBox', 'SvPrdSaf', 'SvStand', 'BgChncFace', 'ExpGAg', 'display'],
      dtype='object')

In [93]:
def percentage_check(x):
    try:
        if '%' in x:
            return float(re.sub('%','',x))
        elif '-' in x:
            return 0.0
    except:
        return x

In [94]:
def norm_line_column(data):
    return round(data / (data.max()+0.05),2)

In [95]:
player_line = defenders_line[defenders_line['name'] == 'Daniel Krutzen'][defenders_line.columns[5:]].copy()
player_line.head()

,Goal,PsOnHfFl,SucflDuels,Touches,DuelLs,SucflTkls,Int,DefTouch,Clrnce,Recovery,TchsM3,Aerials,display
0,0,2,4,118,2,0,0,1,0,6,88,3,Daniel Krutzen
1,0,1,5,103,2,0,2,4,2,7,56,4,Daniel Krutzen
2,0,1,4,96,3,2,1,3,0,5,49,3,Daniel Krutzen
3,0,1,2,95,0,0,2,5,3,3,53,2,Daniel Krutzen
4,0,2,3,94,1,1,1,4,2,7,56,3,Daniel Krutzen


In [96]:
player_line = defenders_line[defenders_line['name'] == 'Daniel Krutzen'][defenders_line.columns[5:]].copy().T
for col in player_line.columns:
    player_line[col] = player_line[col].apply(lambda x: percentage_check(x))
    player_line[col] = norm_line_column(player_line[col])
#player_line = player_line.T

In [97]:
player_line = cpl.index_reset(player_line).values.tolist()
print(player_line)

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.02, 0.01, 0.01, 0.01, 0.02, 0.04, 0.03, 0.07, 0.04, 0.01, 0.04, 0.05, 0.03, 0.06, 0.01, 0.04, 0.06, 0.09, 0.05, 0.1, 0.09, 0.02, 0.04, 0.13, 0.04, 0.09, 0.08, 0.06, 0.0], [0.03, 0.05, 0.04, 0.02, 0.03, 0.04, 0.07, 0.06, 0.01, 0.06, 0.13, 0.05, 0.03, 0.09, 0.06, 0.0, 0.09, 0.08, 0.03, 0.08, 0.09, 0.05, 0.06, 0.1, 0.1, 0.07, 0.08, 0.09, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], [0.02, 0.02, 0.03, 0.0, 0.01, 0.03, 0.12, 0.07, 0.03, 0.1, 0.01, 0.05, 0.0, 0.09, 0.03, 0.03, 0.03, 0.05, 0.03, 0.0, 0.05, 0.04, 0.0, 0.0, 0.06, 0.09, 0.13, 0.06, 0.0], [0.0, 0.0, 0.02, 0.0, 0.01, 0.01, 0.01, 0.02, 0.0, 0.0, 0.05, 0.0, 0.03, 0.03, 0.0, 0.0, 0.0, 0.02, 0.03, 0.02, 0.03, 0.0, 0.02, 0.0, 0.04, 0.0, 0.0, 0.0, 0.0], [0.0, 0.02, 0.01, 0.02, 0.01

In [98]:
player_line = midfielders_line[midfielders_line['name'] == 'Kyle Bekker'][midfielders_line.columns[2:]].copy()
player_line.head()

,Min,ExpG,ExpA,Goal,Off,PsOpHfFl,PsAtt,PsOnHfScs,PsOpHfScs,SucflDuels,Touches,DuelLs,DefTouch,Recovery,TchsA3,TchsM3,PsCmpM3,PsCmpA3,display
21,90,0.07,0.05,0,1,17,91,23,50,2,108,5,5,3,30,69,52,16,Kyle Bekker
22,81,0.00,0.07,0,0,7,89,44,37,2,100,2,2,3,11,69,60,4,Kyle Bekker
23,90,0.03,0.03,0,0,2,73,39,31,2,87,2,5,5,21,54,46,15,Kyle Bekker
24,90,0.03,0.00,0,0,11,74,28,34,1,82,2,3,4,17,50,43,10,Kyle Bekker
25,90,0.05,0.00,0,1,8,66,36,21,5,82,2,3,6,16,54,40,7,Kyle Bekker


In [99]:
player_line = midfielders_line[midfielders_line['name'] == 'Kyle Bekker'][midfielders_line.columns[2:-1]].copy()
for col in player_line.columns:
    player_line[col] = player_line[col].apply(lambda x: percentage_check(x))
    #player_line[col] = norm_line_column(player_line[col])
#player_line = player_line.T

In [100]:
player_line = cpl.index_reset(player_line).values.tolist()
print(player_line)

[[90.0, 0.07, 0.05, 0.0, 1.0, 17.0, 91.0, 23.0, 50.0, 2.0, 108.0, 5.0, 5.0, 3.0, 30.0, 69.0, 52.0, 16.0], [81.0, 0.0, 0.07, 0.0, 0.0, 7.0, 89.0, 44.0, 37.0, 2.0, 100.0, 2.0, 2.0, 3.0, 11.0, 69.0, 60.0, 4.0], [90.0, 0.03, 0.03, 0.0, 0.0, 2.0, 73.0, 39.0, 31.0, 2.0, 87.0, 2.0, 5.0, 5.0, 21.0, 54.0, 46.0, 15.0], [90.0, 0.03, 0.0, 0.0, 0.0, 11.0, 74.0, 28.0, 34.0, 1.0, 82.0, 2.0, 3.0, 4.0, 17.0, 50.0, 43.0, 10.0], [90.0, 0.05, 0.0, 0.0, 1.0, 8.0, 66.0, 36.0, 21.0, 5.0, 82.0, 2.0, 3.0, 6.0, 16.0, 54.0, 40.0, 7.0], [90.0, 0.48, 0.06, 0.0, 3.0, 8.0, 58.0, 19.0, 31.0, 1.0, 73.0, 5.0, 0.0, 2.0, 33.0, 38.0, 30.0, 18.0], [90.0, 0.19, 0.0, 0.0, 1.0, 8.0, 60.0, 19.0, 32.0, 1.0, 72.0, 3.0, 0.0, 2.0, 23.0, 45.0, 37.0, 11.0], [90.0, 0.07, 0.03, 0.0, 1.0, 5.0, 51.0, 12.0, 29.0, 2.0, 70.0, 5.0, 2.0, 8.0, 18.0, 38.0, 26.0, 8.0], [90.0, 0.89, 0.57, 1.0, 0.0, 9.0, 51.0, 15.0, 25.0, 5.0, 68.0, 1.0, 3.0, 5.0, 30.0, 30.0, 23.0, 15.0], [90.0, 0.07, 0.11, 0.0, 1.0, 2.0, 53.0, 26.0, 24.0, 2.0, 68.0, 2.0, 2.0, 6.

In [101]:
try:
    if player_line[0]:
        print(True)
except:
    print(False)

True


In [102]:
test = []
try:
    if test[0]:
        print(True)
except:
    print(False)

False


In [103]:
cols = [x for x in player_game.columns if x not in skip_cols]
cols.remove('week')
cols.remove('Min')
cols.remove('Age')

In [104]:
d_check = player_game[player_game['Position'].isin(position_dict['d'])][cols].describe().T
d_check['common'] = round(d_check['75%'] / d_check['max'],2)
print("'"+"','".join(d_check[d_check['common'] > 0.25].index.values)+"'")

'PsOpHfFl','PsOnHfFl','PsAtt','PsOnHfScs','SucflDuels','Touches','DuelLs','SucflTkls','Int','DefTouch','Clrnce','Recovery','TchsD3','TchsM3','PsAttD3','PsAttM3','PsDist','Aerials'


In [105]:
f_check = player_game[player_game['Position'].isin(position_dict['f'])][cols].describe().T
f_check['common'] = round(f_check['75%'] / f_check['max'],2)
print("'"+"','".join(f_check[f_check['common'] > 0.2].index.values)+"'")

'SubOn','PsOpHfFl','PsAtt','PsOpHfScs','ChncOpnPl','SucflDuels','Touches','DuelLs','Recovery','FlComA3','FlComM3','FlSufM3','BgChncFace','Disposs','TchsA3','TchsM3','TouchOpBox','PsCmpA3','PsDist'


In [106]:
m_check = player_game[player_game['Position'].isin(position_dict['m'])][cols].describe().T
m_check['common'] = round(m_check['75%'] / m_check['max'],2)
print("'"+"','".join(m_check[m_check['common'] > 0.2].index.values)+"'")

'Off','ShotRF','PsOpHfFl','PsOnHfFl','PsAtt','PsOnHfScs','PsOpHfScs','SucflDuels','Touches','DuelLs','DefTouch','Recovery','BgChncFace','TchsA3','TchsD3','TchsM3','PsCmpM3','PsCmpA3','PsAttD3','PsAttM3','PsDist'


In [107]:
k_check = player_game[player_game['Position'].isin(position_dict['k'])][cols].describe().T
k_check['common'] = round(k_check['75%'] / k_check['max'],2)
print("'"+"','".join(k_check[k_check['common'] > 0.1].index.values)+"'")

'PsOpHfFl','PsOnHfFl','PsAtt','PsOnHfScs','PsOpHfScs','Touches','Recovery','GoalCncd','GoalCnIBx','Saves','SvClct','SvDive','SvHands','SvInBox','SvOutBox','SvPrdSaf','SvStand','BgChncFace','TchsD3','PsCmpD3','PsAttD3','PsDist','ExpGAg'


In [108]:
position_dict = {'d':['Right Centre Back','Left Centre Back','Right Back','Central Defender','Right Winger','Left Back','Left Winger','Sub - Defender'],
                 'dc':['team','Age','pos','name','Min','Goal','PsOpHfFl','PsOnHfFl','PsAtt','PsOnHfScs','SucflDuels','Touches','DuelLs','SucflTkls','Int','DefTouch','Clrnce','Recovery','TchsD3','TchsM3','PsAttD3','PsAttM3','PsDist','Aerials'],
                 'f':['Right Winger', 'Left Winger','Centre Forward', 'Second Striker', 'Sub - Attacker'],
                 'fc':['team','Age','pos','name','Min','ExpG','ExpA','Goal','SubOn','PsOpHfFl','PsAtt','PsOpHfScs','ChncOpnPl','SucflDuels','Touches','DuelLs','Recovery','FlComA3','FlComM3','FlSufM3','BgChncFace','Disposs','TchsA3','TchsM3','TouchOpBox','PsCmpA3','PsDist'],
                 'm':['Central Midfielder','Defensive Midfielder','Right Attacking Midfielder','Left Midfielder','Right Midfielder','Centre Attacking Midfielder','Left Attacking Midfielder','Sub - Midfielder'],
                 'mc':['team','Age','pos','name','Min','ExpG','ExpA','Goal','Off','ShotRF','PsOpHfFl','PsOnHfFl','PsAtt','PsOnHfScs','PsOpHfScs','SucflDuels','Touches','DuelLs','DefTouch','Recovery','BgChncFace','TchsA3','TchsD3','TchsM3','PsCmpM3','PsCmpA3','PsAttD3','PsAttM3','PsDist'],
                 'k':['Goalkeeper','Sub - Goalkeeper'],
                 'kc':['team','Age','pos','name','Min','CleanSheet','PsOpHfFl','PsOnHfFl','PsAtt','PsOnHfScs','PsOpHfScs','Touches','Recovery','GoalCncd','GoalCnIBx','Saves','SvClct','SvDive','SvHands','SvInBox','SvOutBox','SvPrdSaf','SvStand','BgChncFace','TchsD3','PsCmpD3','PsAttD3','PsDist','ExpGAg']}